In [1]:
total_count = spark.read.parquet('Files/data/data_total.parquet')
fact_table = spark.read.parquet('Files/data/fact_table.parquet')
user_types = spark.read.parquet('Files/data/user_types.parquet')

StatementMeta(, 415475f9-100a-4796-a1ec-cbae879aface, 3, Finished, Available, Finished)

In [ ]:
user_types.write.mode('overwrite').format('delta').saveAsTable('user_types')

In [6]:
total_count.show(10)

StatementMeta(, c9f17615-7a37-4479-b853-142144a13952, 8, Finished, Available, Finished)

+--------------------+--------------------+--------+----------+---------+
|           real_name|                name|userType|user_count|       id|
+--------------------+--------------------+--------+----------+---------+
|       VT 12 Messilä|               JK IN|       1|    318753|101001874|
|       VT 12 Messilä|              JK OUT|       1|     18493|102001874|
|       VT 12 Messilä|               PP IN|       2|    108557|103001874|
|       VT 12 Messilä|              PP OUT|       2|     52148|104001874|
|Hollolank (Mytäjä...|Hollolank (Mytäjä...|       2|    192719|101002456|
|Harjunkadun_AK-Silta|      JK_Keskustaan2|       1|    333744|101004858|
|Harjunkadun_AK-Silta|        JK_Asemalle2|       1|    347129|102004858|
|Harjunkadun_AK-Silta|      PP_Keskustaan2|       2|    280936|103004858|
|Harjunkadun_AK-Silta|        PP_Asemalle2|       2|    286502|104004858|
|      Ahtialantie_18|       Keskustaan_IN|       1|    142379|101005848|
+--------------------+----------------

In [2]:
total_count_ordered = total_count.select('id', 'real_name', 'name', 'userType', 'user_count')
total_count_ordered.show(5)

StatementMeta(, 8bd7120b-f424-435f-8e79-c71cfb7ca32e, 4, Finished, Available, Finished)

+---------+--------------------+--------------------+--------+----------+
|       id|           real_name|                name|userType|user_count|
+---------+--------------------+--------------------+--------+----------+
|101001874|       VT 12 Messilä|               JK IN|       1|    318753|
|102001874|       VT 12 Messilä|              JK OUT|       1|     18493|
|103001874|       VT 12 Messilä|               PP IN|       2|    108557|
|104001874|       VT 12 Messilä|              PP OUT|       2|     52148|
|101002456|Hollolank (Mytäjä...|Hollolank (Mytäjä...|       2|    192719|
+---------+--------------------+--------------------+--------+----------+
only showing top 5 rows



In [5]:
total_count_ordered = total_count_ordered.drop('real_name')

StatementMeta(, 74704f41-bcac-4cdd-86e4-1f1aac3ac1ef, 7, Finished, Available, Finished)

In [6]:
total_count_ordered = total_count_ordered.withColumnRenamed('id', 'asema_id')

StatementMeta(, 74704f41-bcac-4cdd-86e4-1f1aac3ac1ef, 8, Finished, Available, Finished)

In [7]:
total_count_ordered.show(3)

StatementMeta(, 74704f41-bcac-4cdd-86e4-1f1aac3ac1ef, 9, Finished, Available, Finished)

+---------+------+--------+----------+
| asema_id|  name|userType|user_count|
+---------+------+--------+----------+
|101001874| JK IN|       1|    318753|
|102001874|JK OUT|       1|     18493|
|103001874| PP IN|       2|    108557|
+---------+------+--------+----------+
only showing top 3 rows



In [9]:
from pyspark.sql.functions import col, row_number
from pyspark.sql import Window

window_spec = Window.orderBy('asema_id')
total_count_ordered = total_count_ordered.withColumn('id', row_number().over(window_spec))
#total_count_ordered = total_count_ordered.select('id, asema_id', 'userType', 'user_count')
total_count_ordered.show(5)

StatementMeta(, 74704f41-bcac-4cdd-86e4-1f1aac3ac1ef, 11, Finished, Available, Finished)

+---------+--------------------+--------+----------+---+
| asema_id|                name|userType|user_count| id|
+---------+--------------------+--------+----------+---+
|101001874|               JK IN|       1|    318753|  1|
|101002456|Hollolank (Mytäjä...|       2|    192719|  2|
|101004858|      JK_Keskustaan2|       1|    333744|  3|
|101005848|       Keskustaan_IN|       1|    142379|  4|
|101013931|Uudenmaankatu/Rad...|       2|     45784|  5|
+---------+--------------------+--------+----------+---+
only showing top 5 rows



In [11]:
total_count_ordered = total_count_ordered.select('id', 'asema_id', 'userType', 'user_count')
total_count_ordered.show(3)

StatementMeta(, 74704f41-bcac-4cdd-86e4-1f1aac3ac1ef, 13, Finished, Available, Finished)

+---+---------+--------+----------+
| id| asema_id|userType|user_count|
+---+---------+--------+----------+
|  1|101001874|       1|    318753|
|  2|101002456|       2|    192719|
|  3|101004858|       1|    333744|
+---+---------+--------+----------+
only showing top 3 rows



In [3]:
total_count_ordered.write.mode('overwrite').format('delta').saveAsTable('total_count')

StatementMeta(, 8bd7120b-f424-435f-8e79-c71cfb7ca32e, 5, Finished, Available, Finished)

In [13]:
asemat = total_count.select('id', 'real_name', 'name')
asemat.show(3)

StatementMeta(, 74704f41-bcac-4cdd-86e4-1f1aac3ac1ef, 15, Finished, Available, Finished)

+---------+-------------+------+
|       id|    real_name|  name|
+---------+-------------+------+
|101001874|VT 12 Messilä| JK IN|
|102001874|VT 12 Messilä|JK OUT|
|103001874|VT 12 Messilä| PP IN|
+---------+-------------+------+
only showing top 3 rows



In [15]:
asemat.write.mode('overwrite').format('delta').saveAsTable('asemat')

StatementMeta(, 74704f41-bcac-4cdd-86e4-1f1aac3ac1ef, 17, Finished, Available, Finished)

In [2]:
fact_table.show(5)

StatementMeta(, 3b1faf5f-be13-4aa3-b310-66fda101eb08, 4, Finished, Available, Finished)

+---------+----------+-----------+-----+-----------------+
|       id|       pvm|viikonpäivä|määrä|__index_level_0__|
+---------+----------+-----------+-----+-----------------+
|101001874|2023-01-01|  sunnuntai|   54|                0|
|101001874|2023-01-02|  maanantai|   57|                0|
|101001874|2023-01-03|    tiistai|   70|                0|
|101001874|2023-01-04|keskiviikko|   68|                0|
|101001874|2023-01-05|    torstai|   44|                0|
+---------+----------+-----------+-----+-----------------+
only showing top 5 rows



In [2]:
fact_table = fact_table.drop('__index_level_0__')
fact_table.show(3)

StatementMeta(, 415475f9-100a-4796-a1ec-cbae879aface, 4, Finished, Available, Finished)

+---------+----------+-----------+-----+
|       id|       pvm|viikonpäivä|määrä|
+---------+----------+-----------+-----+
|101001874|2023-01-01|  sunnuntai|   54|
|101001874|2023-01-02|  maanantai|   57|
|101001874|2023-01-03|    tiistai|   70|
+---------+----------+-----------+-----+
only showing top 3 rows



In [3]:
fact_table = fact_table.withColumnRenamed('id', 'asema_id')
fact_table.show(3)

StatementMeta(, 415475f9-100a-4796-a1ec-cbae879aface, 5, Finished, Available, Finished)

+---------+----------+-----------+-----+
| asema_id|       pvm|viikonpäivä|määrä|
+---------+----------+-----------+-----+
|101001874|2023-01-01|  sunnuntai|   54|
|101001874|2023-01-02|  maanantai|   57|
|101001874|2023-01-03|    tiistai|   70|
+---------+----------+-----------+-----+
only showing top 3 rows



In [4]:
from pyspark.sql.functions import col, row_number
from pyspark.sql import Window

window_spec = Window.orderBy('pvm', 'asema_id')
fact_table = fact_table.withColumn('id', row_number().over(window_spec))
fact_table.show(5)

StatementMeta(, 415475f9-100a-4796-a1ec-cbae879aface, 6, Finished, Available, Finished)

+---------+----------+-----------+-----+---+
| asema_id|       pvm|viikonpäivä|määrä| id|
+---------+----------+-----------+-----+---+
|101001874|2023-01-01|  sunnuntai|   54|  1|
|101002456|2023-01-01|  sunnuntai|   64|  2|
|101004858|2023-01-01|  sunnuntai|  193|  3|
|101005848|2023-01-01|  sunnuntai|  131|  4|
|101013931|2023-01-01|  sunnuntai|   40|  5|
+---------+----------+-----------+-----+---+
only showing top 5 rows



In [5]:
fact_table = fact_table.select('id', 'pvm', 'asema_id', 'viikonpäivä', 'määrä')

StatementMeta(, 415475f9-100a-4796-a1ec-cbae879aface, 7, Finished, Available, Finished)

In [6]:
fact_table.show(20)

StatementMeta(, 415475f9-100a-4796-a1ec-cbae879aface, 8, Finished, Available, Finished)

+---+----------+---------+-----------+-----+
| id|       pvm| asema_id|viikonpäivä|määrä|
+---+----------+---------+-----------+-----+
|  1|2023-01-01|101001874|  sunnuntai|   54|
|  2|2023-01-01|101002456|  sunnuntai|   64|
|  3|2023-01-01|101004858|  sunnuntai|  193|
|  4|2023-01-01|101005848|  sunnuntai|  131|
|  5|2023-01-01|101013931|  sunnuntai|   40|
|  6|2023-01-01|101034375|  sunnuntai|   49|
|  7|2023-01-01|101039385|  sunnuntai|   60|
|  8|2023-01-01|101059258|  sunnuntai| 1261|
|  9|2023-01-01|101059446|  sunnuntai|  655|
| 10|2023-01-01|101061358|  sunnuntai|  244|
| 11|2023-01-01|101061359|  sunnuntai|  168|
| 12|2023-01-01|101062896|  sunnuntai|   12|
| 13|2023-01-01|102001874|  sunnuntai|   69|
| 14|2023-01-01|102004858|  sunnuntai|  227|
| 15|2023-01-01|102005848|  sunnuntai|  133|
| 16|2023-01-01|102039385|  sunnuntai|  102|
| 17|2023-01-01|102059258|  sunnuntai| 1507|
| 18|2023-01-01|102059446|  sunnuntai|  710|
| 19|2023-01-01|102061358|  sunnuntai|  224|
| 20|2023-

In [10]:
fact_table.printSchema()

StatementMeta(, 415475f9-100a-4796-a1ec-cbae879aface, 12, Finished, Available, Finished)

root
 |-- id: integer (nullable = false)
 |-- pvm: date (nullable = true)
 |-- asema_id: long (nullable = true)
 |-- viikonpäivä: string (nullable = true)
 |-- määrä: long (nullable = true)



In [9]:
fact_table = fact_table.withColumn('pvm', fact_table['pvm'].cast('date'))

StatementMeta(, 415475f9-100a-4796-a1ec-cbae879aface, 11, Finished, Available, Finished)

In [8]:
print(fact_table.count(), total_count.count())

StatementMeta(, 3b1faf5f-be13-4aa3-b310-66fda101eb08, 10, Finished, Available, Finished)

36825 56


In [11]:
fact_table = fact_table.join(total_count, fact_table.asema_id == total_count.id, 'left').select(fact_table['*'], total_count['userType'])

StatementMeta(, 415475f9-100a-4796-a1ec-cbae879aface, 13, Finished, Available, Finished)

In [10]:
fact_table.count()

StatementMeta(, 3b1faf5f-be13-4aa3-b310-66fda101eb08, 12, Finished, Available, Finished)

36825

In [12]:
fact_table.show(5)

StatementMeta(, 415475f9-100a-4796-a1ec-cbae879aface, 14, Finished, Available, Finished)

+---+----------+---------+-----------+-----+--------+
| id|       pvm| asema_id|viikonpäivä|määrä|userType|
+---+----------+---------+-----------+-----+--------+
|  1|2023-01-01|101001874|  sunnuntai|   54|       1|
|  2|2023-01-01|101002456|  sunnuntai|   64|       2|
|  3|2023-01-01|101004858|  sunnuntai|  193|       1|
|  4|2023-01-01|101005848|  sunnuntai|  131|       1|
|  5|2023-01-01|101013931|  sunnuntai|   40|       2|
+---+----------+---------+-----------+-----+--------+
only showing top 5 rows



In [14]:
missing_vals = fact_table.filter(fact_table['userType'].isNull())
missing_vals.show()

StatementMeta(, 3b1faf5f-be13-4aa3-b310-66fda101eb08, 16, Finished, Available, Finished)

+---+---+--------+-----------+-----+--------+
| id|pvm|asema_id|viikonpäivä|määrä|userType|
+---+---+--------+-----------+-----+--------+
+---+---+--------+-----------+-----+--------+



In [13]:

fact_table.write.mode('overwrite').format('delta').saveAsTable('fact_table')

StatementMeta(, 415475f9-100a-4796-a1ec-cbae879aface, 15, Finished, Available, Finished)

In [1]:
total = spark.read.parquet('Files/data/data_total.parquet')
total.show(3)

StatementMeta(, 4b8db074-9326-45f2-b02b-001d3ce7988f, 3, Finished, Available, Finished)

+-------------+------+--------+----------+---------+
|    real_name|  name|userType|user_count|       id|
+-------------+------+--------+----------+---------+
|VT 12 Messilä| JK IN|       1|    318753|101001874|
|VT 12 Messilä|JK OUT|       1|     18493|102001874|
|VT 12 Messilä| PP IN|       2|    108557|103001874|
+-------------+------+--------+----------+---------+
only showing top 3 rows



In [15]:
fact = spark.read.parquet('Files/data/fact_table.parquet')
fact.show(3)

StatementMeta(, 3b1faf5f-be13-4aa3-b310-66fda101eb08, 17, Finished, Available, Finished)

+---------+----------+-----------+-----+-----------------+
|       id|       pvm|viikonpäivä|määrä|__index_level_0__|
+---------+----------+-----------+-----+-----------------+
|101001874|2023-01-01|  sunnuntai|   54|                0|
|101001874|2023-01-02|  maanantai|   57|                0|
|101001874|2023-01-03|    tiistai|   70|                0|
+---------+----------+-----------+-----+-----------------+
only showing top 3 rows



In [1]:
from datetime import datetime, timedelta
import pandas as pd

StatementMeta(, d08d88fc-121c-4393-9d27-30b2bbf51c6e, 3, Finished, Available, Finished)

In [5]:
start_date = datetime.strptime('2023-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2026-12-31', '%Y-%m-%d')

days = (end_date-start_date).days + 1
df = pd.DataFrame(columns=['Date', 'Year', 'Month_number', 'Month_name', 'Day', 'Weekday_number', 'Weekday_name',\
                           'Quarter', 'Year-Month'] )

kuukaudet = {'January': 'Tammikuu', 'February': 'Helmikuu', 'March': 'Maaliskuu', 'April': 'Huhtikuu',
            'May': 'Toukokuu', 'June': 'Kesäkuu', 'July': 'Heinäkuu', 'August': 'Elokuu',
            'September': 'Syyskuu', 'October': 'Lokakuu', 'November': 'Marraskuu', 'December': 'Joulukuu'}

paivat = {'Monday': 'Maanantai', 'Tuesday': 'Tiistai', 'Wednesday': 'Keskiviikko', 'Thursday': 'Torstai',
          'Friday': 'Perjantai', 'Saturday': 'Lauantai', 'Sunday': 'Sunnuntai'}

for i in range(days):
    xday = start_date + timedelta(i)
    date = datetime.strftime(xday, '%Y-%m-%d')
    year = xday.year
    month_number = xday.month
    print(month_number, xday.month)
    month_name_eng = xday.strftime('%B')
    month_name_fi = kuukaudet[month_name_eng]
    day = xday.day
    weekday_number = xday.weekday()
    weekday_name_eng = xday.strftime('%A')
    weekday_name_fi = paivat[weekday_name_eng]
    quarter = pd.Timestamp(date).quarter
    year_month = xday.strftime('%Y-%m')
    new_row = pd.DataFrame({'Date': [date], 'Year': [year], 'Month_number': [month_number], 'Month_name': [month_name_fi],
                           'Day': [day], 'Weekday_number': [weekday_number], 'Weekday_name': [weekday_name_fi],
                           'Quarter': [quarter], 'Year-Month': [year_month]})
    df = pd.concat([df, new_row], ignore_index=True)

StatementMeta(, d08d88fc-121c-4393-9d27-30b2bbf51c6e, 7, Finished, Available, Finished)

1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
9 9
9 9
9 9
9 9
9 9
9 9
9 9


In [6]:
print(len(df))
df.head()

StatementMeta(, d08d88fc-121c-4393-9d27-30b2bbf51c6e, 8, Finished, Available, Finished)

778


,Date,Year,Month_number,Month_name,Day,Weekday_number,Weekday_name,Quarter,Year-Month
0,2023-01-01,2023,1,Tammikuu,1,6,Sunnuntai,1,2023-01
1,2023-01-02,2023,1,Tammikuu,2,0,Maanantai,1,2023-01
2,2023-01-03,2023,1,Tammikuu,3,1,Tiistai,1,2023-01
3,2023-01-04,2023,1,Tammikuu,4,2,Keskiviikko,1,2023-01
4,2023-01-05,2023,1,Tammikuu,5,3,Torstai,1,2023-01


In [7]:
df['Date'] = pd.to_datetime(df['Date'])
df.dtypes

StatementMeta(, d08d88fc-121c-4393-9d27-30b2bbf51c6e, 9, Finished, Available, Finished)

Date              datetime64[ns]
Year                      object
Month_number              object
Month_name                object
Day                       object
Weekday_number            object
Weekday_name              object
Quarter                   object
Year-Month                object
dtype: object

In [8]:
df.tail(3)

StatementMeta(, d08d88fc-121c-4393-9d27-30b2bbf51c6e, 10, Finished, Available, Finished)

,Date,Year,Month_number,Month_name,Day,Weekday_number,Weekday_name,Quarter,Year-Month
775,2025-02-14,2025,2,Helmikuu,14,4,Perjantai,1,2025-02
776,2025-02-15,2025,2,Helmikuu,15,5,Lauantai,1,2025-02
777,2025-02-16,2025,2,Helmikuu,16,6,Sunnuntai,1,2025-02


In [9]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

schema = StructType([
    StructField('Date', DateType(), True),\
    StructField('Year', IntegerType(), True),\
    StructField('Month_number', IntegerType(), True),\
    StructField('Month_name', StringType(), True),\
    StructField('Day', IntegerType(), True),\
    StructField('Weekday_number', IntegerType(), True),\
    StructField('Weekday_name', StringType(), True),\
    StructField('Quarter', IntegerType(), True),\
    StructField('Year-Month', StringType(), True)
])

StatementMeta(, d08d88fc-121c-4393-9d27-30b2bbf51c6e, 11, Finished, Available, Finished)

In [10]:
spark_df = spark.createDataFrame(df, schema=schema)

StatementMeta(, d08d88fc-121c-4393-9d27-30b2bbf51c6e, 12, Finished, Available, Finished)

In [11]:
spark_df.show(5)

StatementMeta(, d08d88fc-121c-4393-9d27-30b2bbf51c6e, 13, Finished, Available, Finished)

+----------+----+------------+----------+---+--------------+------------+-------+----------+
|      Date|Year|Month_number|Month_name|Day|Weekday_number|Weekday_name|Quarter|Year-Month|
+----------+----+------------+----------+---+--------------+------------+-------+----------+
|2023-01-01|2023|           1|  Tammikuu|  1|             6|   Sunnuntai|      1|   2023-01|
|2023-01-02|2023|           1|  Tammikuu|  2|             0|   Maanantai|      1|   2023-01|
|2023-01-03|2023|           1|  Tammikuu|  3|             1|     Tiistai|      1|   2023-01|
|2023-01-04|2023|           1|  Tammikuu|  4|             2| Keskiviikko|      1|   2023-01|
|2023-01-05|2023|           1|  Tammikuu|  5|             3|     Torstai|      1|   2023-01|
+----------+----+------------+----------+---+--------------+------------+-------+----------+
only showing top 5 rows



In [12]:
spark_df.printSchema()

StatementMeta(, d08d88fc-121c-4393-9d27-30b2bbf51c6e, 14, Finished, Available, Finished)

root
 |-- Date: date (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month_number: integer (nullable = true)
 |-- Month_name: string (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Weekday_number: integer (nullable = true)
 |-- Weekday_name: string (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Year-Month: string (nullable = true)



In [13]:
spark_df.write.mode('overwrite').format('delta').saveAsTable('date_table')

StatementMeta(, d08d88fc-121c-4393-9d27-30b2bbf51c6e, 15, Finished, Available, Finished)